In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. URL de la página
url = "https://es.wikipedia.org/wiki/Anexo:Resultados_del_Informe_PISA_en_Espa%C3%B1a_por_Comunidades_Aut%C3%B3nomas#2022"

# 2. Realizar la solicitud
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# 3. Localizar el encabezado h3 con id="2022"
h3_2022 = soup.find("h3", {"id": "2022"})

# 4. Encontrar la tabla con border="1" que aparece justo después del h3 de 2022
table_2022 = h3_2022.find_next("table", {"border": "1"})

# 5. Dentro de esa tabla, hallar sus subtablas con border="0" (3 subtablas para las 3 habilidades)
subtables = table_2022.find_all("table", {"border": "0"})

# Para etiquetar cada subtabla con el nombre de la habilidad
habilidades_map = {
    0: "Habilidad lectora",
    1: "Matemáticas",
    2: "Ciencias"
}

# Aquí guardamos todas las filas (sin limpiar ni transformar)
data = []

# 6. Recorrer las subtablas y sus filas
for i, subtable in enumerate(subtables):
    habilidad = habilidades_map.get(i, f"Habilidad_{i}")
    
    rows = subtable.find_all("tr")
    for row in rows:
        # Obtenemos todas las celdas
        cells = row.find_all('td')
        # La estructura esperada: 3 celdas (Posición / Comunidad / Puntuación)
        if len(cells) >= 3:
            # a) Posición (sin limpiar ni convertir a número)
            posicion = cells[0].get_text(strip=True)
            
            # b) Comunidad Autónoma
            #    Algunas celdas tienen 2 enlaces (bandera y nombre). Tomamos el último <a>
            all_links = cells[1].find_all("a", href=True)
            if all_links:
                comunidad = all_links[-1].get_text(strip=True)
            else:
                comunidad = cells[1].get_text(strip=True)
            
            # c) Puntuación (sin convertir a float)
            puntuacion = cells[2].get_text(strip=True)
            
            # Añadimos la fila con su habilidad
            data.append([posicion, comunidad, puntuacion, habilidad])

# 7. Crear el DataFrame final con los datos en bruto
df = pd.DataFrame(data, columns=["Posición", "Comunidad Autónoma", "Puntuación", "Habilidad"])

# 8. Mostrar el DataFrame (sin limpieza ni transformación)
df



,Posición,Comunidad Autónoma,Puntuación,Habilidad
0,1.,Castilla y León,498,Habilidad lectora
1,2.,Asturias,497,Habilidad lectora
2,3.,Comunidad de Madrid,496,Habilidad lectora
3,4.,Cantabria,494,Habilidad lectora
4,5.,Aragón,488,Habilidad lectora
5,6.,La Rioja,487,Habilidad lectora
6,7.,Galicia,485,Habilidad lectora
7,8.,Comunidad Valenciana,482,Habilidad lectora
8,9.,Navarra,478,Habilidad lectora
9,,Media OCDE,476,Habilidad lectora


In [68]:
df["Puntuación"] = df["Puntuación"].astype(int)

# Hacemos el pivot. Usamos `aggfunc='first'` para evitar error si hay filas duplicadas.
result_df = df.pivot_table(
    index='Comunidad Autónoma',
    columns='Habilidad',
    values='Puntuación',
    aggfunc='first'
).reset_index()

print("=== Formato pivotado (tabular), incluyendo promedios y totales ===")
result_df




=== Formato pivotado (tabular), incluyendo promedios y totales ===


Habilidad,Comunidad Autónoma,Ciencias,Habilidad lectora,Matemáticas
0,Andalucía,473,461,457
1,Aragón,499,488,487
2,Asturias,503,497,495
3,Cantabria,504,494,495
4,Castilla y León,506,498,499
5,Castilla-La Mancha,475,468,464
6,Cataluña,477,462,469
7,Ceuta,410,404,395
8,Comunidad Valenciana,483,482,473
9,Comunidad de Madrid,502,496,494


In [69]:

result_df.columns.name = None
result_df

,Comunidad Autónoma,Ciencias,Habilidad lectora,Matemáticas
0,Andalucía,473,461,457
1,Aragón,499,488,487
2,Asturias,503,497,495
3,Cantabria,504,494,495
4,Castilla y León,506,498,499
5,Castilla-La Mancha,475,468,464
6,Cataluña,477,462,469
7,Ceuta,410,404,395
8,Comunidad Valenciana,483,482,473
9,Comunidad de Madrid,502,496,494


In [70]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Comunidad Autónoma  22 non-null     object
 1   Ciencias            22 non-null     int64 
 2   Habilidad lectora   22 non-null     int64 
 3   Matemáticas         22 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 836.0+ bytes


In [71]:
# Guardamos como csv
result_df.to_csv("datos/resultados_pisa_2022_sin_limpieza.csv", index=False)

In [72]:
df_resultados_pisa = pd.read_csv("datos/resultados_pisa_2022_sin_limpieza.csv")

In [73]:
df_resultados_pisa

,Comunidad Autónoma,Ciencias,Habilidad lectora,Matemáticas
0,Andalucía,473,461,457
1,Aragón,499,488,487
2,Asturias,503,497,495
3,Cantabria,504,494,495
4,Castilla y León,506,498,499
5,Castilla-La Mancha,475,468,464
6,Cataluña,477,462,469
7,Ceuta,410,404,395
8,Comunidad Valenciana,483,482,473
9,Comunidad de Madrid,502,496,494
